In [14]:
# | default_exp vitdet3dmae

# MAE using VitDet3D architecture for encoding
#### (and simple ViT for decoding)

- [Exploring Plain Vision Transformer Backbones for Object Detection](https://arxiv.org/abs/2203.16527)
- [Uniform Masking: Enabling MAE Pre-training for Pyramid-based Vision Transformers with Locality](https://arxiv.org/abs/2205.10063)
- [Swin MAE: Masked Autoencoders for Small Datasets](https://arxiv.org/abs/2212.13805)

Structure of code:
(Recommended extension: https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid)

```mermaid
flowchart TB
    subgraph ViTDet3DMAEModel
        subgraph ViTDet3DMAEEmbeddings
            subgraph ViTDet3DMAEPatchEmbeddings
                patch_embeddings[Patchify scan]
            end
            positional_embeddings[Add Positional Embeddings]
            uniform_sampling[Uniform Sampling]
            masking[Mask]
        end
        ViTDet3DMAEEncoder
    end

    Start -- 3D input scan --> patch_embeddings
    patch_embeddings --> positional_embeddings
    positional_embeddings --> uniform_sampling
    positional_embeddings -- Embeddings --> ViTDet3DMAEEncoder
    uniform_sampling -- 25% mask / 12.5% mask --> masking
    masking --> ViTDet3DMAEEncoder

    ViTDet3DMAEEncoder --> End
```

## Imports

In [15]:
# | export

import torch
import numpy as np
import collections

from copy import deepcopy
from dataclasses import dataclass
from einops import rearrange
from medct.vitdet3d import VitDetConfig, VitDet3dEncoder, VitDet3dPreTrainedModel
from torch import nn
from transformers.models.vit_mae.modeling_vit_mae import ViTMAELayer
from transformers.utils import ModelOutput

## Config

In [16]:
from transformers.models.vit_mae import ViTMAEConfig

ViTMAEConfig()


ViTMAEConfig {
  "attention_probs_dropout_prob": 0.0,
  "decoder_hidden_size": 512,
  "decoder_intermediate_size": 2048,
  "decoder_num_attention_heads": 16,
  "decoder_num_hidden_layers": 8,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "mask_ratio": 0.75,
  "model_type": "vit_mae",
  "norm_pix_loss": false,
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.36.0"
}

In [51]:
# | export
# Config inherits hyperparameters from both ViTDet and ViTMAE
class ViTDet3DMAEConfig(VitDetConfig):
    def __init__(
        self,
        attention_probs_dropout_prob=0.0,
        decoder_hidden_size=384,
        decoder_intermediate_size=1536,
        decoder_learnable_position_embeddings=False,
        decoder_num_attention_heads=16,
        decoder_num_hidden_layers=8,
        hidden_dropout_prob=0.0,
        hidden_size=768,
        image_size=(64, 512, 512),
        intermediate_size=3072,
        learnable_position_embeddings=False,
        mask_ratio=0.875,
        norm_pix_loss=False,
        patch_size=(2, 32, 32),
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)

        self.model_type = "vitdet3dmae"

        self.attention_probs_dropout_prob = attention_probs_dropout_prob
        self.decoder_hidden_size = decoder_hidden_size
        self.decoder_intermediate_size = decoder_intermediate_size
        self.decoder_learnable_position_embeddings = decoder_learnable_position_embeddings
        self.decoder_num_attention_heads = decoder_num_attention_heads
        self.decoder_num_hidden_layers = decoder_num_hidden_layers
        self.hidden_dropout_prob = hidden_dropout_prob
        self.hidden_size = hidden_size
        self.image_size = image_size
        self.intermediate_size = intermediate_size
        self.learnable_position_embeddings = learnable_position_embeddings
        self.mask_ratio = mask_ratio
        self.norm_pix_loss = norm_pix_loss
        self.patch_size = patch_size

        self.grid_size = (
            self.image_size[0] // self.patch_size[0],
            self.image_size[1] // self.patch_size[1],
            self.image_size[2] // self.patch_size[2],
        )
        self.num_patches = self.grid_size[0] * self.grid_size[1] * self.grid_size[2]

In [52]:
ViTDet3DMAEConfig()


ViTDet3DMAEConfig {
  "attention_probs_dropout_prob": 0.0,
  "decoder_hidden_size": 384,
  "decoder_intermediate_size": 1536,
  "decoder_learnable_position_embeddings": false,
  "decoder_num_attention_heads": 16,
  "decoder_num_hidden_layers": 8,
  "drop_path_rate": 0.0,
  "dropout_prob": 0.0,
  "grid_size": [
    32,
    16,
    16
  ],
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": [
    64,
    512,
    512
  ],
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-06,
  "learnable_position_embeddings": false,
  "mask_ratio": 0.875,
  "mlp_ratio": 4,
  "model_type": "vitdet",
  "norm_pix_loss": false,
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "num_patches": 8192,
  "out_features": [
    "stage12"
  ],
  "out_indices": [
    12
  ],
  "patch_size": [
    2,
    32,
    32
  ],
  "pretrain_image_size": 224,
  "qkv_bias": true,
  "residual_block_indices": [],
  "stage_names": [

In [53]:
sample_config = ViTDet3DMAEConfig(
    image_size=(32, 128, 128),
    patch_size=(2, 16, 32),  # num patches = 16*8*4 = 512
    num_channels=1,
    mask_ratio=0.75,  # or set it to 0.875
)

# Embeddings

#### Position embeddings

In [54]:
# | export
def get_3d_position_embeddings(embedding_size, grid_size):
    if embedding_size % 6 != 0:
        raise ValueError("embed_dim must be divisible by 6")

    d, h, w = grid_size

    grid_d = np.arange(d, dtype=np.float32)
    grid_h = np.arange(h, dtype=np.float32)
    grid_w = np.arange(w, dtype=np.float32)

    grid = np.meshgrid(grid_w, grid_h, grid_d)
    grid = np.stack(grid, axis=0)

    grid = grid.reshape([3, 1, d, h, w])

    omega = np.arange(embedding_size // 6, dtype=float)
    omega /= embedding_size / 6.0
    omega = 1.0 / 10000**omega

    position_embeddings = []
    for grid_subset in grid:
        grid_subset = grid_subset.reshape(-1)
        out = np.einsum("m,d->md", grid_subset, omega)

        emb_sin = np.sin(out)
        emb_cos = np.cos(out)

        emb = np.concatenate([emb_sin, emb_cos], axis=1)
        position_embeddings.append(emb)

    position_embeddings = np.concatenate(position_embeddings, axis=1)

    return position_embeddings

#### Patch embeddings

In [55]:
# | export
class ViTDet3DMAEPatchEmbeddings(nn.Module):
    def __init__(self, config: ViTDet3DMAEConfig) -> None:
        super().__init__()

        image_size, patch_size = config.image_size, config.patch_size
        num_channels, hidden_size = config.num_channels, config.hidden_size

        if not isinstance(image_size, collections.abc.Iterable) or len(image_size) != 3:
            raise ValueError("image_size must be given as 3D iterable")
        if not isinstance(patch_size, collections.abc.Iterable) or len(patch_size) != 3:
            raise ValueError("patch_size must be given as 3D iterable")

        self.image_size = image_size
        self.patch_size = patch_size
        self.num_channels = num_channels

        self.projection = nn.Conv3d(num_channels, hidden_size, kernel_size=patch_size, stride=patch_size)

    def forward(self, pixel_values: torch.Tensor):
        num_channels = pixel_values.shape[1]
        if num_channels != self.num_channels:
            raise ValueError(
                "Make sure that the channel dimension of the pixel values match with the one set in the configuration."
                f" Expected {self.num_channels} but got {num_channels}."
            )

        if self.image_size != pixel_values.shape[2:]:
            raise ValueError(
                "Make sure that the spatial dimensions of the pixel values match with the ones set in the configuration."
                f" Expected {self.image_size} but got {pixel_values.shape[2:]}."
            )

        # (b, c, z, y, x)
        embeddings = self.projection(pixel_values)
        # (b, hidden_size, z, y, x)

        return embeddings

In [56]:
sample_scan = torch.rand(1, 1, 32, 128, 128)

output = ViTDet3DMAEPatchEmbeddings(sample_config)(sample_scan)

assert np.prod(output.shape[2:]) == 512

output.shape

torch.Size([1, 768, 16, 8, 4])

##### Overall embeddings

In [57]:
# | export
class ViTDet3DMAEEmbeddings(nn.Module):
    def __init__(self, config: ViTDet3DMAEConfig):
        super().__init__()

        self.config = config

        self.patch_embeddings = ViTDet3DMAEPatchEmbeddings(config)

        # Positional embeddings
        num_positions = self.config.num_patches  # Not adding class token as the patches can't be flattened in ViTDet3D
        self.position_embeddings = nn.Parameter(
            torch.zeros(1, config.hidden_size, num_positions), requires_grad=config.learnable_position_embeddings
        )
        self.initialize_weights()

    def initialize_weights(self):
        if not self.config.learnable_position_embeddings:
            # initialize (and freeze) position embeddings by sin-cos embedding
            position_embeddings = get_3d_position_embeddings(self.config.hidden_size, self.config.grid_size)
            self.position_embeddings.data.copy_(torch.from_numpy(position_embeddings.T).float().unsqueeze(0))

        # initialize patch_embeddings like nn.Linear (instead of nn.Conv2d)
        w = self.patch_embeddings.projection.weight.data
        torch.nn.init.xavier_uniform_(w.view([w.shape[0], -1]))

    def uniform_sampling(self, embeddings: torch.Tensor):
        # embeddings: (b, hidden_size, z, y, x)
        _, _, Z, Y, X = embeddings.shape
        assert Z % 2 == 0 and Y % 2 == 0 and X % 2 == 0, "Z, Y, X must be even"

        # Only works if mask_ratio is 0.75 or 0.875
        assert self.config.mask_ratio in {0.75, 0.875}, "Mask ratio must be 0.75 or 0.875"

        masks = []
        for _ in range(len(embeddings)):
            # Perform uniform sampling. Select one patch in a 2x2x2 cube.
            # Also add the diagonally opposite patch if mask_ratio == 0.75.
            num_decisions = X * Y * Z // 8
            if self.config.mask_ratio == 0.75:
                mask1 = np.eye(4, dtype="int").repeat(num_decisions, axis=0)  # (num_decisions * 4, 4)
                np.random.shuffle(mask1)
                mask1 = mask1[:num_decisions]  # (num_decisions, 4)
                mask2 = mask1.copy()

                mask2[:, (0, 3)] = mask2[:, (3, 0)]
                mask2[:, (1, 2)] = mask2[:, (2, 1)]
                mask = np.concatenate([mask1, mask2], axis=1)  # (num_decisions, 8)
            else:
                mask = np.eye(8, dtype="int").repeat(num_decisions, axis=0)  # (num_decisions * 8, 8)
                np.random.shuffle(mask)
                mask = mask[:num_decisions]  # (num_decisions, 8)

            mask = rearrange(
                mask, "(d h w) (p1 p2 p3) -> (d p1) (h p2) (w p3)", d=Z // 2, h=Y // 2, w=X // 2, p1=2, p2=2, p3=2
            )
            masks.append(mask)

        masks = np.stack(masks, axis=0)
        masks = torch.tensor(masks, device=embeddings.device, dtype=torch.int8)

        return masks

    def forward(self, pixel_values: torch.Tensor):
        # Get patch embeddings
        embeddings = self.patch_embeddings(pixel_values)
        # (b, hidden_size, z, y, x)

        # Add positional embeddings
        B, C, Z, Y, X = embeddings.shape
        embeddings = embeddings + self.position_embeddings.reshape(1, C, Z, Y, X)

        # Uniform sampling
        masks = self.uniform_sampling(embeddings)

        # Get masked embeddings
        new_Z, new_Y, new_X = Z // 2, Y // 2, X // 2
        if self.config.mask_ratio == 0.75:
            new_Z = Z
        embeddings = torch.masked_select(embeddings, masks.unsqueeze(1).bool()).reshape(B, C, new_Z, new_Y, new_X)

        return embeddings, masks

In [58]:
sample_scan = torch.rand(2, 1, 32, 128, 128)

embeddings, masks = ViTDet3DMAEEmbeddings(sample_config)(sample_scan)

embeddings.shape, masks.shape

(torch.Size([2, 768, 16, 4, 2]), torch.Size([2, 16, 8, 4]))

In [59]:
display(masks[0, :2, :4, :4])
display(masks[1, :2, :4, :4])


tensor([[[0, 1, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
         [1, 0, 0, 0]],

        [[0, 0, 1, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 1]]], dtype=torch.int8)


tensor([[[1, 0, 0, 1],
         [0, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 1, 0]],

        [[0, 0, 0, 0],
         [0, 1, 1, 0],
         [0, 0, 0, 1],
         [1, 0, 0, 0]]], dtype=torch.int8)

#### Visualize a scan

In [26]:
from qer.ai.core.preprocessing.load_scan import load_scan
from neuro_utils.visualize import plot_scans
import SimpleITK as sitk

/home/users/arjun.agarwal/miniconda3/lib/python3.9/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)
/home/users/arjun.agarwal/miniconda3/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [27]:
scan = load_scan(r"/home/users/arjun.agarwal/projects/temp/1.2.840.113619.2.81.290.1.3689.20170516.222854")
scan = sitk.GetArrayFromImage(scan)
scan = np.clip(scan, 0, 80)
scan = np.pad(scan, ((12, 12), (0, 0), (0, 0)))
scan = torch.from_numpy(scan).float()
scan = scan.unsqueeze(0).unsqueeze(0)
scan.shape

torch.Size([1, 1, 64, 512, 512])

In [28]:
_, masks = ViTDet3DMAEEmbeddings(ViTDet3DMAEConfig(num_channels=1))(scan)
masks.shape

torch.Size([1, 32, 16, 16])

In [29]:
expanded_masks = masks.repeat_interleave(2, dim=1).repeat_interleave(32, dim=2).repeat_interleave(32, dim=3)
expanded_masks.shape

torch.Size([1, 64, 512, 512])

In [30]:
expanded_masks.shape

torch.Size([1, 64, 512, 512])

In [31]:
scan = scan.squeeze(0).squeeze(0)
masked_scan = scan * expanded_masks.squeeze(0)
plot_scans([scan, masked_scan])

interactive(children=(IntSlider(value=0, description='z', max=63), Output()), _dom_classes=('widget-interact',…

# Model

In [60]:
# | export
class ViTDet3DMAEEncoder(VitDet3dEncoder):
    pass

In [61]:
# | export
@dataclass
class ViTDet3DMAEModelOutput(ModelOutput):
    last_hidden_state: torch.FloatTensor = None
    masks: torch.LongTensor = None
    hidden_states: tuple[torch.FloatTensor] = None
    attentions: tuple[torch.FloatTensor] = None

/home/users/arjun.agarwal/miniconda3/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [62]:
# | export
class ViTDet3DMAEPreTrainedModel(VitDet3dPreTrainedModel):
    config_class = ViTDet3DMAEConfig

    def _init_weights(self, module):
        super()._init_weights(module)

        if isinstance(module, ViTDet3DMAEEmbeddings):
            module.position_embeddings.data = nn.init.trunc_normal_(
                module.position_embeddings.data.to(torch.float32),
                mean=0.0,
                std=self.config.initializer_range,
            ).to(module.position_embeddings.dtype)

In [63]:
# | export
class ViTDet3DMAEModel(ViTDet3DMAEPreTrainedModel):
    def __init__(self, config: ViTDet3DMAEConfig):
        super().__init__(config)

        self.config = config

        self.embeddings = ViTDet3DMAEEmbeddings(config)
        self.encoder = ViTDet3DMAEEncoder(config)

        # Initialize weights and apply final processing
        self.post_init()

    def forward(
        self,
        pixel_values: torch.FloatTensor = None,
        head_mask: torch.FloatTensor = None,
        output_attentions: bool = None,
        output_hidden_states: bool = None,
        return_dict: bool = None,
    ):
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if pixel_values is None:
            raise ValueError("You have to specify pixel_values")

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embedding_output, masks = self.embeddings(pixel_values)

        encoder_outputs = self.encoder(
            embedding_output,
            head_mask=head_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = encoder_outputs[0]

        if not return_dict:
            return (sequence_output,) + encoder_outputs[1:]

        return ViTDet3DMAEModelOutput(
            last_hidden_state=sequence_output,
            masks=masks,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
        )

In [64]:
sample_scan = torch.rand(2, 1, 32, 128, 128)

model = ViTDet3DMAEModel(sample_config)
output = model(sample_scan)

output.last_hidden_state.shape, output.masks.shape

(torch.Size([2, 768, 16, 4, 2]), torch.Size([2, 16, 8, 4]))

In [65]:
sum([x.numel() for x in model.parameters()])

86234880

# Decoder

In [66]:
# | export
@dataclass
class ViTDet3DMAEDecoderOutput(ModelOutput):
    logits: torch.FloatTensor
    hidden_states: tuple[torch.FloatTensor] = None
    attentions: tuple[torch.FloatTensor] = None

/home/users/arjun.agarwal/miniconda3/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [67]:
# | export
class ViTDet3DMAEDecoder(nn.Module):
    def __init__(self, config: ViTDet3DMAEConfig, grid_size: tuple[int, int, int]):
        super().__init__()
        self.config = config

        self.decoder_embed = nn.Linear(config.hidden_size, config.decoder_hidden_size, bias=True)
        self.mask_token = nn.Parameter(torch.zeros(1, config.decoder_hidden_size, 1, 1, 1))
        self.position_embeddings = nn.Parameter(
            torch.zeros(1, config.decoder_hidden_size, grid_size[0] * grid_size[1] * grid_size[2]),
            requires_grad=config.decoder_learnable_position_embeddings,
        )

        decoder_config = deepcopy(config)
        decoder_config.hidden_size = config.decoder_hidden_size
        decoder_config.num_hidden_layers = config.decoder_num_hidden_layers
        decoder_config.num_attention_heads = config.decoder_num_attention_heads
        decoder_config.intermediate_size = config.decoder_intermediate_size
        self.decoder_layers = nn.ModuleList(
            [ViTMAELayer(decoder_config) for _ in range(config.decoder_num_hidden_layers)]
        )

        self.decoder_norm = nn.LayerNorm(config.decoder_hidden_size, eps=config.layer_norm_eps)
        self.decoder_pred = nn.Linear(
            config.decoder_hidden_size, np.prod(config.patch_size) * config.num_channels, bias=True
        )  # encoder to decoder
        self.gradient_checkpointing = False
        self.config = config
        self.initialize_weights(grid_size)

    def initialize_weights(self, grid_size):
        if not self.config.decoder_learnable_position_embeddings:
            # initialize (and freeze) position embeddings by sin-cos embedding
            position_embeddings = get_3d_position_embeddings(self.config.decoder_hidden_size, grid_size)
            self.position_embeddings.data.copy_(torch.from_numpy(position_embeddings.T).float().unsqueeze(0))

        # timm's trunc_normal_(std=.02) is effectively normal_(std=0.02) as cutoff is too big (2.)
        torch.nn.init.normal_(self.mask_token, std=self.config.initializer_range)

    def forward(
        self,
        hidden_states,
        masks,
        output_attentions=False,
        output_hidden_states=False,
        return_dict=True,
    ):
        # embed tokens
        (B, C, nD, nH, nW) = hidden_states.shape
        hidden_states = hidden_states.permute(0, 2, 3, 4, 1).reshape(-1, C)
        x = self.decoder_embed(hidden_states)
        x = x.reshape(B, nD, nH, nW, self.config.decoder_hidden_size)
        x = x.permute(0, 4, 1, 2, 3)

        # Create mask tokens array
        D, H, W = nD * 2, nH * 2, nW * 2
        if self.config.mask_ratio == 0.75:
            D = nD
        mask_tokens = self.mask_token.repeat(B, 1, D, H, W)

        # Add hidden states to this
        x = mask_tokens.masked_scatter(masks.unsqueeze(1).bool(), x)

        # add pos embed
        hidden_states = x + self.position_embeddings.reshape(1, -1, D, H, W)

        # Flatten hidden states and reorder them
        hidden_states = hidden_states.reshape(B, self.config.decoder_hidden_size, -1)
        hidden_states = hidden_states.permute(0, 2, 1)

        # apply Transformer layers (blocks)
        all_hidden_states = () if output_hidden_states else None
        all_self_attentions = () if output_attentions else None
        for i, layer_module in enumerate(self.decoder_layers):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            if self.gradient_checkpointing and self.training:
                layer_outputs = self._gradient_checkpointing_func(
                    layer_module.__call__,
                    hidden_states,
                    None,
                    output_attentions,
                )
            else:
                layer_outputs = layer_module(hidden_states, head_mask=None, output_attentions=output_attentions)

            hidden_states = layer_outputs[0]

            if output_attentions:
                all_self_attentions = all_self_attentions + (layer_outputs[1],)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        hidden_states = self.decoder_norm(hidden_states)

        # predictor projection
        logits = self.decoder_pred(hidden_states)
        logits = logits.permute(0, 2, 1)

        # Reshaping back to image size
        logits = logits.reshape(B, -1, D, H, W)
        # p1, p2, p3 = self.config.patch_size
        # logits = rearrange(
        #     logits,
        #     "b (c p1 p2 p3) nD nH nW -> b c (nD p1) (nH p2) (nW p3)",
        #     c=self.config.num_channels,
        #     p1=p1,
        #     p2=p2,
        #     p3=p3,
        # )  # This reshapes the logits to the original pixel_values shape

        if not return_dict:
            return tuple(v for v in [logits, all_hidden_states, all_self_attentions] if v is not None)

        return ViTDet3DMAEDecoderOutput(
            logits=logits,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
        )

In [68]:
decoder = ViTDet3DMAEDecoder(sample_config, (16, 8, 4))
decoded = decoder(output.last_hidden_state, output.masks)

decoded.logits.shape

torch.Size([2, 1024, 16, 8, 4])

# ViTDet3DMAEForPreTraining

In [69]:
# | export
@dataclass
class ViTDet3DMAEForPreTrainingOutput(ModelOutput):
    loss: torch.FloatTensor = None
    logits: torch.FloatTensor = None
    masks: torch.LongTensor = None
    hidden_states: tuple[torch.FloatTensor] = None
    attentions: tuple[torch.FloatTensor] = None

/home/users/arjun.agarwal/miniconda3/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [74]:
# | export
class ViTDet3DMAEForPreTraining(ViTDet3DMAEPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.config = config

        self.vitdet3d = ViTDet3DMAEModel(config)
        self.decoder = ViTDet3DMAEDecoder(config, grid_size=config.grid_size)

        # Initialize weights and apply final processing
        self.post_init()

    def forward_loss(self, pixel_values, pred, masks_inverse):
        B, C, Z, Y, X = pixel_values.shape
        nD, nH, nW = self.config.grid_size
        D, H, W = self.config.patch_size

        target = pixel_values.reshape(B, C, nD, D, nH, H, nW, W)
        target = rearrange(target, "b c nD D nH H nW W -> b c nD nH nW D H W")
        target = target.reshape(*pred.shape)

        loss = (pred - target) ** 2
        loss = loss.mean(dim=1)  # [N, L], mean loss per patch

        loss = (loss * masks_inverse).sum() / masks_inverse.sum()  # mean loss on removed patches
        return loss

    def forward(
        self,
        pixel_values: torch.FloatTensor = None,
        head_mask: torch.FloatTensor = None,
        output_attentions: bool = None,
        output_hidden_states: bool = None,
        return_dict: bool = None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.vitdet3d(
            pixel_values,
            head_mask=head_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        latent = outputs.last_hidden_state
        masks = outputs.masks

        decoder_outputs = self.decoder(latent, masks)
        logits = decoder_outputs.logits

        loss = self.forward_loss(pixel_values, logits, 1 - masks)

        if not return_dict:
            output = (logits, masks) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return ViTDet3DMAEForPreTrainingOutput(
            loss=loss,
            logits=logits,
            masks=masks,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [75]:
model = ViTDet3DMAEForPreTraining(sample_config)
output = model(sample_scan)

output.loss, output.logits.shape, output.masks.shape

(tensor(0.4998, grad_fn=<DivBackward0>), torch.Size([2, 1024, 16, 8, 4]), torch.Size([2, 16, 8, 4]))

# Overfitting test

In [76]:
from tqdm.auto import tqdm

In [77]:
sample_batch = torch.rand(5, 1, 32, 128, 128)
sample_config = ViTDet3DMAEConfig(
    decoder_hidden_size=96,
    decoder_intermediate_size=384,
    decoder_num_attention_heads=2,
    decoder_num_hidden_layers=2,
    hidden_size=192,
    intermediate_size=768,
    num_attention_heads=4,
    num_hidden_layers=4,
    #
    image_size=(32, 128, 128),
    patch_size=(4, 32, 32),
    num_channels=1,
    mask_ratio=0.875,
)

model = ViTDet3DMAEForPreTraining(sample_config)

sum(x.numel() for x in model.parameters())

3242752

In [78]:
optimizer = torch.optim.SGD(model.parameters(), lr=1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [79]:
sample_batch = sample_batch.cuda()
model = model.cuda()

In [80]:
for i in tqdm(range(100)):
    optimizer.zero_grad()
    output = model(sample_batch)
    print(f"Loss: {output.loss:f}\tLR: {scheduler.get_last_lr()[0]:f}")
    output.loss.backward()
    optimizer.step()
    scheduler.step()

  0%|          | 0/100 [00:00<?, ?it/s]

Loss: 0.377929	LR: 1.000000
Loss: 0.355454	LR: 1.000000
Loss: 0.334494	LR: 1.000000
Loss: 0.315483	LR: 1.000000
Loss: 0.297968	LR: 1.000000
Loss: 0.281640	LR: 0.900000
Loss: 0.267757	LR: 0.900000
Loss: 0.254762	LR: 0.900000
Loss: 0.242536	LR: 0.900000
Loss: 0.231009	LR: 0.900000
Loss: 0.220180	LR: 0.810000
Loss: 0.211066	LR: 0.810000
Loss: 0.202335	LR: 0.810000
Loss: 0.194160	LR: 0.810000
Loss: 0.186445	LR: 0.810000
Loss: 0.179131	LR: 0.729000
Loss: 0.172979	LR: 0.729000
Loss: 0.167132	LR: 0.729000
Loss: 0.161721	LR: 0.729000
Loss: 0.156476	LR: 0.729000
Loss: 0.151539	LR: 0.656100
Loss: 0.147430	LR: 0.656100
Loss: 0.143555	LR: 0.656100
Loss: 0.139819	LR: 0.656100
Loss: 0.136296	LR: 0.656100
Loss: 0.132875	LR: 0.590490
Loss: 0.130188	LR: 0.590490
Loss: 0.127571	LR: 0.590490
Loss: 0.125022	LR: 0.590490
Loss: 0.122547	LR: 0.590490
Loss: 0.120340	LR: 0.531441
Loss: 0.118487	LR: 0.531441
Loss: 0.116557	LR: 0.531441
Loss: 0.114839	LR: 0.531441
Loss: 0.113183	LR: 0.531441
Loss: 0.111592	LR: 0

# Export

In [81]:
!nbdev_export